The purpose of this notebook is to take the initial model produced by model_train.ipynb and use the same functionality to train over more labelled records. This will build on the weights that are currently set in the model and further tune them by exposing the model to more data.

You should be able to run this over and over again and see better and better results on the test dataset. 

In [ ]:
import pandas as pd
import torch 
import numpy as np
from data_helpers import feature_prep, split_sample
from model import load_ckp, BERTClass, train_model

## Load in Best Model

Set the path to your favorite model here. Note that models are saved to a warm_start folder to prevent confusion.

In [ ]:
load_model = "best_model.pt"

Now let's set our device.

In [ ]:
if torch.cuda.is_available(): # check for CUDA gpu
    device = torch.device("cuda")
elif torch.backends.mps.is_available(): # Check for Apple M1/M2 chip
    device = torch.device("mps")
else:
    device = torch.device("cpu") # Otherwise just use CPU

**NB:** Once the optimizer has been through a few epochs the learning rate is so small that no significant improvements are made in the results. To rectify this I have been reseting the optimizer here rather than using the one loaded from the checkpoint.

In [ ]:
EPOCHS = 4
LEARNING_RATE = 1e-05

model = BERTClass()
model.to(device)
optimizer_init = torch.optim.Adam()
checkpoint_path = "warm_start/current_checkpoint.pt"
best_model = "warm_start/best_model.pt"
valid_loss_min_input = np.Inf
model, optimizer, epoch, valid_loss_min_input = load_ckp(load_model, model, optimizer_init)

## Data Sample

The next few cells we will take a cut of the remaining unseen observations ready for training and remove those recrods from the unseen data to avoid picking them out again.

In [ ]:
import json

with open("observation_categories.json", "r") as f:
    categories = json.load(f)["categories"]

**NB:** observation_categories.json and observations_unseen.csv should have been created by model_train.ipynb

In [ ]:
model_data = pd.read_csv("observations-unseen.csv")
print(f"FULL Dataset: {model.shape}")
display(model_data.head())

We can now take a sample of the unseen data that we have loaded in for training our notebook.

In [ ]:
SAMPLE_SIZE = 50000
TRAIN_SIZE = 0.8

sample_data = model_data.sample(SAMPLE_SIZE)